In [1]:
import os
from datamodule import ExampleDataModule

In [ ]:
class Hparams:
    def __init__(self, datadir, img_shape, vol_shape, batch_size, seed) -> None:
        self.datadir = datadir
        self.img_shape = img_shape
        self.vol_shape = vol_shape
        self.batch_size = batch_size
        self.seed = seed

# hparams = parser.parse_args()
hparams = Hparams(
    datadir='/home/quantm/data',
    img_shape=256, 
    vol_shape=256, 
    batch_size=4, 
    seed=42
)

In [ ]:
train_image3d_folders = [os.path.join(hparams.datadir, "Visualization/image3d"),]
train_image2d_folders = [os.path.join(hparams.datadir, "Visualization/image2d"),]
val_image3d_folders = [os.path.join(hparams.datadir, "Visualization/image3d"),]
val_image2d_folders = [os.path.join(hparams.datadir, "Visualization/image2d"),]
test_image3d_folders = [os.path.join(hparams.datadir, "Visualization/image3d"),]
test_image2d_folders = [os.path.join(hparams.datadir, "Visualization/image2d"),]

datamodule = ExampleDataModule(
    train_image3d_folders=train_image3d_folders,
    train_image2d_folders=train_image2d_folders,
    val_image3d_folders=val_image3d_folders,
    val_image2d_folders=val_image2d_folders,
    test_image3d_folders=test_image3d_folders,
    test_image2d_folders=test_image2d_folders,
    img_shape=hparams.img_shape,
    vol_shape=hparams.vol_shape,
    batch_size=hparams.batch_size,
)
datamodule.setup(seed=hparams.seed)


In [ ]:
import torch
from nerfstudio.cameras.cameras import Cameras, CameraType
from nerfstudio.utils import plotly_utils as vis
import plotly.graph_objects as go 

In [ ]:
# cx = 20.0
# cy = 10.0
# fx = 20.0
# fy = 20.0

# c2w = torch.eye(4)[None, :3, :]

# camera = Cameras(fx=fx, fy=fy, cx=cx, cy=cy, camera_to_worlds=c2w, camera_type=CameraType.PERSPECTIVE)
# fig = vis.vis_camera_rays(camera)
# fig.show()

In [ ]:
# num_samples = 3
# near_plane = 1
# far_plane = 3

# ray_bundle = camera.generate_rays(camera_indices=0)

# bins = torch.linspace(near_plane, far_plane, num_samples + 1)[..., None]
# ray_samples = ray_bundle.get_ray_samples(bin_starts=bins[:-1, :], bin_ends=bins[1:, :])

# vis_rays = vis.get_ray_bundle_lines(ray_bundle, color="teal", length=far_plane)

# fig = go.Figure(
#     data=[
#         vis_rays, 
#         vis.get_frustum_points(ray_samples.frustums)
#     ], 
#     # layout=webdocs_layout
# )
# fig.show()

In [ ]:
# from nerfstudio.model_components import renderers

# num_samples = 10

# rgb_samples = torch.ones((3, num_samples, 3))
# weights = torch.ones((3, num_samples, 1))
# weights /= torch.sum(weights, dim=-2, keepdim=True)

# rgb_renderer = renderers.RGBRenderer()

# rgb = rgb_renderer(rgb=rgb_samples, weights=weights)


In [3]:
from typing import Literal, Optional, Tuple

import torch
from torch import Tensor, nn

from nerfstudio.cameras.rays import RayBundle, RaySamples
from nerfstudio.data.scene_box import SceneBox
from nerfstudio.field_components.activations import trunc_exp
from nerfstudio.fields.base_field import Field
from nerfstudio.models.base_model import Model, ModelConfig

from nerfstudio.model_components.ray_samplers import ProposalNetworkSampler, UniformSampler
from nerfstudio.model_components.renderers import AccumulationRenderer, DepthRenderer, NormalsRenderer, RGBRenderer
from nerfstudio.model_components.scene_colliders import NearFarCollider
from nerfstudio.model_components.shaders import NormalsShader

class ScalarField(Field):
    def __init__(
        self,
        aabb: Tensor,
        implementation: Literal["tcnn", "torch"] = "torch",
    ) -> None:
        super().__init__()
        self.register_buffer("aabb", aabb)

    def get_density(self, ray_samples: RaySamples) -> Tuple[Tensor, None]:
        positions = SceneBox.get_normalized_positions(ray_samples.frustums.get_positions(), self.aabb)
        # Make sure the tcnn gets inputs between 0 and 1.
        selector = ((positions > 0.0) & (positions < 1.0)).all(dim=-1)
        positions = positions * selector[..., None]
        positions_flat = positions.view(-1, 3)

        density = density * selector[..., None]
        return density, None

    def get_outputs(self, ray_samples: RaySamples, density_embedding: Optional[Tensor] = None) -> dict:
        return {}
    
class ScalarModel(Model):
    config: ModelConfig

    def populate_modules(self):
        """Set the fields and modules."""
        super().populate_modules()
        # scene_contraction = SceneContraction(order=float("inf"))

        # Fields
        self.field = ScalarField()
        
        # Samplers
        self.sampler = UniformSampler(single_jitter=self.config.use_single_jitter)
        # Colliders

        # Renderers
        self.renderer_rgb = RGBRenderer(background_color=self.config.background_color)
        
        # Losses

        # Metrics

    def get_outputs(self, ray_bundle: RayBundle):
        # apply the camera optimizer pose tweaks
        if self.training:
            self.camera_optimizer.apply_to_raybundle(ray_bundle)
        ray_samples: RaySamples
        ray_samples, weights_list, ray_samples_list = self.proposal_sampler(ray_bundle, density_fns=self.density_fns)
        field_outputs = self.field.forward(ray_samples, compute_normals=self.config.predict_normals)
        if self.config.use_gradient_scaling:
            field_outputs = scale_gradients_by_distance_squared(field_outputs, ray_samples)

        weights = ray_samples.get_weights(field_outputs[FieldHeadNames.DENSITY])
        weights_list.append(weights)
        ray_samples_list.append(ray_samples)

        rgb = self.renderer_rgb(rgb=field_outputs[FieldHeadNames.RGB], weights=weights)
        
        outputs = {
            "rgb": rgb,
        }
        return outputs    
    
scalar_modle = ScalarModel(
    config=ModelConfig,
    scene_box=SceneBox, 
    num_train_data=1,
)

AttributeError: type object 'ModelConfig' has no attribute 'collider_params'